# Eastern Washington Digital Equity

## Nicholas Tran

# Preparation

## Import The Modules

In [1]:
import numpy as np  # matrix and array manipulation
import pandas as pd  # dataframe manipulation
import plotly.express as px

# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from pingouin import cronbach_alpha
from scipy.stats import pearsonr
from sklearn import linear_model as lm
from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler  # scale the data
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from factor_analyzer.factor_analyzer import (
    calculate_kmo,
)  # get measure of sampling adequacy

# initialize the scaler
scaler = StandardScaler()


# use this as a method in corr() to get the pearson p values
def pearsonr_pval(x, y):
    return pearsonr(x, y)[1]


# turn scientific notation into decimals
pd.options.display.float_format = "{:.10f}".format

## Import The Dataset

In [654]:
dataset1 = pd.read_csv("../app/data/combined_data.csv")

dataset = dataset1.drop(
    columns=[
        "GEOID",
        "tract",
        "access_with_no_subscription",
        "has_computer",
        'foreign_born',
        'desktop_or_laptop',
        'internet_subscription',
        'naturalized_citizen',
        'desktop_or_laptop_only',
        'broadband',
        'smartphone',
        'dial_up',
        'satellite',
        'mean_income',
        "other_internet_service",
        'tablet_or_portable',
        "not_citizen"
    ]
)


dataset.head()

,native_citizen,work_from_home,smartphone_only,no_computer,no_internet_access,sixty_five_and_older,median_income,mean_d_mbps,mean_u_mbps,mean_lat_ms,number_providers,lowest_cost
0,2484,82,153,76,137,505,52589.0000000000,62.6586413043,9.5285652174,42.6413043478,12.0000000000,30.0000000000
1,1623,47,68,64,116,228,59886.0000000000,61.9772073171,9.0375243902,60.9024390244,12.0000000000,30.0000000000
2,1064,3,76,183,163,259,39928.0000000000,88.0868093023,13.2451162791,60.1674418605,8.0000000000,30.0000000000
3,1669,26,132,55,88,258,58884.0000000000,87.0107878788,17.0365959596,44.4242424242,8.0000000000,30.0000000000
4,1841,110,97,24,66,271,50915.0000000000,74.1417594937,14.2204050633,37.8924050633,8.0000000000,30.0000000000


## Create A Class for The Data

In [655]:
class PCA:
    """Input a df and get many things back.
    https://stackoverflow.com/questions/13224362/principal-component-analysis-pca-in-python
    """

    def __init__(self, df):
        import numpy as np
        from scipy import linalg as LA

        self.data = df

        # scale data
        self.scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

        # kmo, total kmo
        self.kmo, self.total_kmo = calculate_kmo(self.scaled)
        self.kmo = pd.DataFrame(self.data.columns, self.kmo).reset_index()
        self.kmo = self.kmo.rename(
            columns={"index": "KMO", 0: "Variables"}
        ).sort_values("KMO")

        # center data
        self.center = self.scaled.apply(lambda x: x - x.mean())

        # covariance
        self.cov = pd.DataFrame(
            np.cov(self.center, rowvar=False),
            columns=self.scaled.columns,
            index=self.scaled.columns,
        )

        # eigenvalues and loadings(eigenvectors)
        self.eigenvalues, self.loadings = LA.eigh(self.cov)

        # sort eigenvalues and loadings from
        sorter = np.argsort(self.eigenvalues)[::-1]
        self.loadings = self.loadings[:, sorter]
        self.eigenvalues = self.eigenvalues[sorter]

        pc_list = ["pc" + str(i + 1) for i in range(len(self.eigenvalues))]

        # turn into dataframe
        self.loadings = pd.DataFrame(
            self.loadings, index=self.scaled.columns, columns=pc_list
        )
        self.eigenvalues = pd.DataFrame(
            self.eigenvalues, index=pc_list, columns=["eigenvalues"]
        )

        # pca scores - scaled data * loadings
        self.scores = self.scaled @ self.loadings

        # percent explained
        explained_variance = self.eigenvalues / self.eigenvalues.sum() * 100
        self.percent_explained = pd.DataFrame(explained_variance).round(2)

        self.percent_explained[
            "cumulative_explained_variance"
        ] = self.percent_explained.cumsum().round(2)
        self.percent_explained.columns.values[0] = "explained_variance"

        # scree plot
        self.scree = (
            px.line(
                self.percent_explained,
                x=pc_list,
                y="cumulative_explained_variance",
                text="cumulative_explained_variance",
                color=px.Constant("cumulative explained variance"),
            )
            .update_traces(textposition="top left")
            .add_bar(
                x=pc_list,
                y=self.percent_explained.explained_variance,
                name="explained variance",
                text=self.percent_explained.explained_variance,
            )
        )

    def calculate_weights(self, number_of_components):
        """calculate coefficients using your eigenvalues. Multiplies each row by the respective
        eigenvalue. Row 1 of loadings will be multiplied by eigenvalue 1. Row 2 with 2. You may
        only use this when you have at least 2 pcs.

        Args:
            number_of_components (int): number of pcs you want to use.

        Returns:
            DataFrame: Returns a dataframe of weights.
        """
        weights = (
            self.loadings.iloc[:, 0:number_of_components]
            .mul(
                [
                    float(self.eigenvalues.iloc[i, :])
                    for i in range(len(self.eigenvalues))
                ],
                axis=0,
            )
            .sum(axis=1)
        )
        return weights


data = PCA(dataset)
data.weights = data.calculate_weights(4)

data.scaled["index"] = data.scaled @ data.weights
data.weights = (
    pd.DataFrame(data.weights).reset_index().rename(columns={0: "coefficients"})
)

In [656]:
data.percent_explained

,explained_variance,cumulative_explained_variance
pc1,23.2900000000,23.2900000000
pc2,23.0200000000,46.3100000000
pc3,19.0600000000,65.3700000000
pc4,11.3500000000,76.7200000000
pc5,6.0700000000,82.7900000000
pc6,4.1100000000,86.9000000000
pc7,3.7600000000,90.6600000000
pc8,2.8100000000,93.4700000000
pc9,2.5400000000,96.0100000000
pc10,1.7100000000,97.7200000000


In [657]:
data.eigenvalues


,eigenvalues
pc1,2.8103353057
pc2,2.7771437275
pc3,2.2991932861
pc4,1.3694354633
pc5,0.7328469667
pc6,0.4962142965
pc7,0.4542766901
pc8,0.3389923433
pc9,0.3068837557
pc10,0.2058173008


In [658]:
data.scree


In [659]:
# X = all the independent variables
X = data.scaled.iloc[:, :-1]

# y = index, or dependent variable
y = data.scaled.iloc[:, -1]


In [660]:
lr = lm.LinearRegression()

sfs = SFS(
    lr,
    n_features_to_select="auto",
    n_jobs=-1,
)

# sfs = SFS(
#     lr,
#     k_features="parsimonious",
#     verbose=0,
#     forward=True,
#     scoring="r2",  # picks model on r2
#     cv=5,
#     n_jobs=-1,
# )

sfs.fit(X, y)
selected_variables = list(sfs.get_feature_names_out())

In [661]:
selected_variables = list(sfs.get_feature_names_out())

In [662]:
selected_variables

['native_citizen',
 'work_from_home',
 'smartphone_only',
 'no_computer',
 'mean_d_mbps',
 'mean_u_mbps']

In [663]:
X_new = data.scaled[selected_variables]

In [664]:
model = lr
model.fit(X_new, y)

LinearRegression()

In [665]:
model.feature_names_in_

array(['native_citizen', 'work_from_home', 'smartphone_only',
       'no_computer', 'mean_d_mbps', 'mean_u_mbps'], dtype=object)

In [666]:
model.coef_

array([-1.00625826,  0.54414789, -1.10777882, -0.72389806, -0.43758194,
       -0.43927279])

In [667]:
r2 = model.score(X_new, y)
observations = X_new.shape[0]
predictors = X_new.shape[1]
adj_r2 = 1 - (1 - r2) * (observations - 1) / (observations - predictors - 1)
print(f"adjr2: {adj_r2}")


adjr2: 0.9861842002882262


In [668]:
# results = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
# results


In [669]:
# largest_before_1 = results[results["avg_score"] != 1].tail(1).index.to_list()
# largest_before_1 = largest_before_1[0] - 1
# model_vars = list(results.iloc[largest_before_1, 3])
# model_vars

In [670]:
# model = lr
# model.fit(final_X, final_y)
# r2 = model.score(final_X, final_y)
# observations = final_X.shape[0]
# predictors = final_X.shape[1]
# adj_r2 = 1 - (1 - r2) * (observations - 1) / (observations - predictors - 1)
# print(f"adjr2: {adj_r2}")


In [671]:
fin_df = pd.DataFrame(-model.coef_, model.feature_names_in_)
fin_df["vif"] = [VIF(X_new.values, i) for i in range(X_new.shape[1])]
fin_df = fin_df.rename(columns={0: "coefficient"})
fin_df


,coefficient,vif
native_citizen,1.0062582573,1.7969580995
work_from_home,-0.5441478866,1.6481037403
smartphone_only,1.1077788178,1.4080240003
no_computer,0.7238980568,1.2106073440
mean_d_mbps,0.4375819369,1.2593099817
mean_u_mbps,0.4392727919,1.2362992391


In [672]:
coefficients = -model.coef_
final_vars = list(model.feature_names_in_)
final_data = data.scaled[final_vars]
final_data["index"] = final_data.mul(coefficients).sum(axis=1)

C:\Users\Nicholas\AppData\Local\Temp\ipykernel_3392\1205995849.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [673]:
final_data

,native_citizen,work_from_home,smartphone_only,no_computer,mean_d_mbps,mean_u_mbps,index
0,-0.7793647066,-0.5115079191,0.5485540934,-0.3341503606,-1.6263184336,-0.9375212194,-1.2635955438
1,-1.4274799394,-0.8676949403,-0.4644767653,-0.4867380487,-1.6343848222,-0.9561168609,-2.9663187568
2,-1.8482655551,-1.3154729099,-0.3691326845,1.0264231913,-1.3253157117,-0.7967759956,-1.7398485202
3,-1.3928535739,-1.0814071531,0.2982758812,-0.6011788147,-1.3380529792,-0.6531932364,-1.7903312967
4,-1.2633810767,-0.2265583020,-0.1188544723,-0.9953636755,-1.4903884747,-0.7598419576,-2.9861576904
...,...,...,...,...,...,...,...
178,0.2112504459,-1.0407000649,-0.0354284016,-0.6647570181,0.1229907201,2.9177987183,1.5939322068
179,-0.1320022209,0.0990984031,-0.7743450279,0.2380534696,-0.8491271755,0.9769717497,-0.8146346929
180,0.0622065248,-0.8269878522,-0.3333786542,0.4923662831,-1.8089348075,0.2263970815,-0.1923944133
181,0.2534042822,0.2313964396,-0.1903625329,0.5305132051,-0.3223962253,4.5747370603,2.1707169206


In [674]:
cronbach_alpha(final_data)[0]

0.670469086375667

In [675]:
final_data

,native_citizen,work_from_home,smartphone_only,no_computer,mean_d_mbps,mean_u_mbps,index
0,-0.7793647066,-0.5115079191,0.5485540934,-0.3341503606,-1.6263184336,-0.9375212194,-1.2635955438
1,-1.4274799394,-0.8676949403,-0.4644767653,-0.4867380487,-1.6343848222,-0.9561168609,-2.9663187568
2,-1.8482655551,-1.3154729099,-0.3691326845,1.0264231913,-1.3253157117,-0.7967759956,-1.7398485202
3,-1.3928535739,-1.0814071531,0.2982758812,-0.6011788147,-1.3380529792,-0.6531932364,-1.7903312967
4,-1.2633810767,-0.2265583020,-0.1188544723,-0.9953636755,-1.4903884747,-0.7598419576,-2.9861576904
...,...,...,...,...,...,...,...
178,0.2112504459,-1.0407000649,-0.0354284016,-0.6647570181,0.1229907201,2.9177987183,1.5939322068
179,-0.1320022209,0.0990984031,-0.7743450279,0.2380534696,-0.8491271755,0.9769717497,-0.8146346929
180,0.0622065248,-0.8269878522,-0.3333786542,0.4923662831,-1.8089348075,0.2263970815,-0.1923944133
181,0.2534042822,0.2313964396,-0.1903625329,0.5305132051,-0.3223962253,4.5747370603,2.1707169206


In [676]:
final_data = dataset1[["GEOID", "tract"]].join(final_data)

In [677]:
# final_data.to_csv("../app/data/index_data4.csv", index=False)